# Matrix Factorization （矩阵分解）
## 简介

矩阵分解在推荐系统中有广泛的应用。在许多网站，比如IMDB，Netfix等，用户都会给电影一个评分。评分的高低决定了用户对这部电影的喜爱程度，很显然分数高的电影会得到更多未来用户的青睐，又比如说某一用户钟爱某一类型的电影（比如说动作片），那么这个用户就会更加倾向于观看其他的动作片。推荐系统就是解决如何给用户推荐他们喜欢的电影。在本篇中，我主要介绍一种简单的矩阵分解的方法。

  矩阵分解，顾名思义，就是把矩阵分解成两个或两个以上的矩阵。这里需要被分解的矩阵是用户对电影的评分矩阵,记作$\mathbf{R} \in \mathbb{R}^{ U \times D}$, 其中$U$是用户的个数，而$D$是电影的个数，如下我们给出一个评分矩阵$\mathbf{R}$。评分矩阵中，5分表示用户评价相应电影最高的分数，而0分则表示用户并未观看过这部电影。很显然这样的评分矩阵一定是一个非常稀疏的矩阵。
  
  | - | 异形1 | 蜘蛛侠1 | 拯救大兵瑞恩 | 阳光灿烂的日子 | 阿凡达 | 我不是药神 | 无间道1 | 黑鹰坠落 |
|----------|-------|---------|--------------|----------------|--------|------------|---------|----------|
| 爱因斯坦 | 5 | 0 | 3 | 0 | 4 | 0 | 0 | 2 |
| 李白 | 0 | 0 | 0 | 3 | 0 | 5 | 4 | 0 |
| 高斯 | 0 | 4 | 0 | 0 | 4 | 0 | 0 | 2 |
| 牛顿 | 5 | 4 | 1 | 0 | 5 | 0 | 0 | 0 |
| 华盛顿 | 0 | 1 | 5 | 0 | 2 | 0 | 0 | 5 |
| 鲁迅 | 4 | 3 | 4 | 2 | 0 | 5 | 5 | 3 |
| 曹雪芹 | 0 | 0 | 0 | 4 | 0 | 5 | 5 | 0 |
  
  而分解的效果往往发现用户或者电影的隐藏特征（latent features）。我们把一个用户用向量$\mathbf{p} \in \mathbb{R}^{K}$来表征,其中$K$代表特征的个数。用户的特征可以包括，对动作片的喜爱程度，对某个电影演员的偏爱，对某个导演的偏爱等等，当然在矩阵分解实际操作中，这些特征往往都是涵义模糊的。我们希望把评分矩阵分解为如下的形式
  
$$ \tag{1}
\mathbf{R} \approx \mathbf{P} \times \mathbf{Q}^T = \mathbf{\hat{R}} .
$$ 

其中用户矩阵$\mathbf{P} \in \mathbb{R}^{U \times K}$, 电影矩阵$\mathbf{Q} \in \mathbb{R}^{D \times K}$。电影矩阵中的每一行$\mathbf{q}$代表电影在隐藏特征的相关度。 我们分解的目的是使得评分矩阵$\mathbf{R}$中非零元素和期望矩阵$\mathbf{\hat{R}}$中相应的元素之间的误差最小。原本评分矩阵的零元素是用户未曾观看的电影，而在期望矩阵中其相应的元素是用户对该电影的评分期望。显然，在推荐系统中，我们会推荐那些期望评分高的电影。

为了方便之后的讨论，我们把公式（1）写的更为明白
$$ \tag{2}
\mathbf{R} = 
\left[
\begin{matrix}
 r_{11}      & r_{12}      & \cdots & r_{1D}      \\
 r_{21}      & r_{22}      & \cdots & r_{2D}      \\
 \vdots & \vdots & \ddots & \vdots \\
 r_{U1}      & r_{U2}      & \cdots & r_{UD}      \\
\end{matrix}
\right]
\approx
\left[
\begin{matrix}
 \mathbf{p}_1      \\
 \mathbf{p}_2      \\
 \vdots  \\
 \mathbf{p}_U      \\
\end{matrix}
\right]
\times
\left[
\begin{matrix}
 \mathbf{q}_1      \\
 \mathbf{q}_2      \\
 \vdots  \\
 \mathbf{q}_U      \\
\end{matrix}
\right]^T。
$$
期望矩阵中的各个分量可以写作
$$\tag{3}
\hat{r}_{ij} = \mathbf{p}_i \times \mathbf{q}_j^T = \sum_{k = 1}^{K} p_{ik} q_{jk}。
$$


我们把评分矩阵中的非零元素的集合表示为$\cal{K}$,我们的目标是最小化如下的式子
$$ \tag{4}
E = \sum_{r_{ij} \in \cal K} (r_{ij} - \hat{r}_{ij})^2 + \frac{\lambda}{2} \sum_{u=1}^{U}(||\mathbf{p}_u||^2 + ||\mathbf{q}_u||^2).
$$
其中公式的后半部分是规则化的部分，$\lambda$为规则化参数，这是为了防止我们过拟合数据。

## 梯度下降法

我们选用梯度下降法来对公式（4）进行求解局域最小话的方案。对于迭代的方案我们有如下式：
$$ \tag{5}
p'_{ik} = p_{ik} - \gamma \frac{\partial E}{\partial p_{ik}}, \\
q'_{jk} = q_{jk} - \gamma \frac{\partial E}{\partial q_{jk}}.
$$
其中$\gamma$是学习速率，我们一共需要求解$K\times(U+D)$个未知的参数。公式（5）中的的偏导数表示为，

$$ \tag{6}
\frac{\partial E}{\partial p_{ik}} = -2(r_{ij} - \hat{r}_{ij})q_{jk} + \lambda p_{ik}, \\
\frac{\partial E}{\partial q_{jk}} = -2(r_{ij} - \hat{r}_{ij})p_{ik} + \lambda q_{jk}.
$$

我们把公式（6）代入公式（5），得到如下的迭代关系，
$$\tag{7}
p'_{ik} = (1-\gamma \lambda)p_{ik} + 2 \gamma (r_{ij} - \hat{r}_{ij})q_{jk}, \\
q'_{jk} = (1-\gamma \lambda)q_{jk} + 2 \gamma (r_{ij} - \hat{r}_{ij})p_{ik}.
$$

有了这样的迭代公式后，我们就基本上大功搞成了。在实际操作中，评分矩阵$\mathbf{R}$是一个很大的矩阵，所以分解这样的大型矩阵肯定需要比较好的可以并行的算法，具体操作还需要参考一些文献。

In [12]:
import numpy as np
# rating matrix
R = [
    [5, 0, 3, 0, 4, 0, 0, 2],
    [0, 0, 0, 3, 0, 5, 4, 0],
    [0, 4, 0, 0, 4, 0, 0, 2],
    [5, 4, 1, 0, 5, 0, 0, 0],
    [0, 1, 5, 0, 2, 0, 0, 5],
    [4, 3, 4, 2, 0, 5, 5, 3],
    [0, 0, 0, 4, 0, 5, 5, 0]
]

R = np.array(R)
U = len(R)
D = len(R[0])

K = 3

P = numpy.random.rand(U,K)
Q = numpy.random.rand(D,K)

In [45]:
def matrix_factorization(R, P, Q, K, steps = 5000, gamma=0.0002, lamb = 0.02):
    
    for _ in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:], Q[j,:])
                    for k in range(K):
                        # EQ. (7)
                        P[i][k] = (1 - gamma*lamb)*P[i][k] + 2*gamma*eij*Q[j][k]
                        Q[j][k] = (1 - gamma*lamb)*Q[j][k] + 2*gamma*eij*P[i][k]
        eR = numpy.dot(P, Q.T)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:], Q[j,:])
                    e += eij**2
                    for k in range(K):
                        e = e + (lamb/2.0) * (P[i][k]**2 + Q[j][k])**2
        if e < 0.001:
            break
    return P, Q
    

In [47]:
eP, eQ = matrix_factorization(R, P, Q, K)
eR = numpy.dot(eP, eQ.T)

In [48]:
eR

array([[4.98827027, 4.30061905, 2.98343224, 3.43156318, 3.97305082,
        5.22420392, 4.53962588, 2.01011235],
       [4.46767567, 3.97605465, 2.96803422, 2.97251845, 3.35103458,
        4.96066186, 4.03591301, 1.62212986],
       [4.65818069, 3.87369826, 2.01676215, 3.48980984, 4.11210213,
        4.26048268, 4.08426516, 1.95847649],
       [4.98662952, 4.07499476, 1.01050123, 4.17979517, 4.89545976,
        3.72704535, 3.95963644, 1.92077762],
       [2.78035719, 1.02864828, 4.97941395, 0.42977788, 1.98503069,
        4.32453495, 5.75251706, 4.95861236],
       [3.98930061, 2.97896121, 3.99342191, 2.05147373, 2.91229586,
        4.9908056 , 4.930865  , 3.0391567 ],
       [5.41248131, 4.37755567, 2.53877541, 3.96207548, 4.79689915,
        5.00661753, 4.99062986, 2.59219205]])

  | - | 异形1 | 蜘蛛侠1 | 拯救大兵瑞恩 | 阳光灿烂的日子 | 阿凡达 | 我不是药神 | 无间道1 | 黑鹰坠落 |
|----------|-------|---------|--------------|----------------|--------|------------|---------|----------|
| 爱因斯坦 | 4.99 | 4.30 | 2.98 | 3.43 | 3.97 | 5.22 | 4.539 | 2.01 |
| 李白 | 4.46 | 3.97 | 2.96 | 2.97 | 3.35 | 4.96 | 4.04 | 1.62 |
| 高斯 | 4.65 | 3.87 | 2.02 | 3.48 | 4.11 | 4.26 | 4.08 | 1.95 |
| 牛顿 | 4.97 | 4.07 | 1.01 | 4.17 | 4.89 | 3.72 | 3.96 | 1.92 |
| 华盛顿 | 2.78 | 1.02 | 4.98 | 0.42 | 1.98 | 4.32 | 5.75 | 4.96 |
| 鲁迅 | 3.99 | 2.98 | 3.99 | 2.05 | 2.91 | 4.99 | 4.93 | 3.03 |
| 曹雪芹 | 5.41 | 4.37 | 2.54 | 3.96 | 4.79 | 5.00 | 4.99 | 2.59 |